In [ ]:
"""
This module is used to pull data from twitter API and send data to
Spark Streaming process using socket. It acts like a client of
twitter API and a server of spark streaming. It open a listening TCP
server socket, and listen to any connection from TCP client. After
a connection established, it send streaming data to it.


Usage:
  If used with dataproc:
    gcloud dataproc jobs submit pyspark --cluster <Cluster Name> twitterHTTPClient.py

  Make sure that you run this module before you run spark streaming process.
  Please remember stop the job on dataproc if you no longer want to stream data.

Todo:
  1. change the credentials to your own

"""

import json
import socket

from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

# credentials
# TODO: replace with your own credentials
# 
ACCESS_TOKEN = '73924326-MqBTht1798PhhBFBbZVXYvs4nDvFjVfMkrrbTHwkB'  # your access token
ACCESS_SECRET = 'P7j3nlzC5ofw37radLfASIAEHSkrYkRQZavfWCfvuvmoS'  # your access token secret
CONSUMER_KEY = 'DT8HiPRRPcsxfoqnYxMqacnMc'  # your API key
CONSUMER_SECRET = 'WpBQgvRYbMZDNj3bhQok5hVLKTGCKwQHeRTI16FdDjEQ3uqHYn'  # your API secret key

# the tags to track
tags = ['#', 'bigdata', 'spark', 'ai', 'movie', 'coronavirus']


class TweetsListener(StreamListener):
    """
    tweets listener object
    """

    def __init__(self, csocket):
        super(TweetsListener, self).__init__()
        self.client_socket = csocket

    def on_data(self, data):
        try:
            msg = json.loads(data)
            print('TEXT:{}\n'.format(msg['text']))
            self.client_socket.send(msg['text'].encode('utf-8'))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            return False
        # return True

    def on_error(self, status):
        print(status)
        return False


def sendData(c_socket, tags):
    """
    send data to socket
    """
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=tags, languages=['en'])


class twitter_client:
    def __init__(self, TCP_IP, TCP_PORT):
        self.s = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.s.bind((TCP_IP, TCP_PORT))

    def run_client(self, tags):
        try:
            self.s.listen(1)
            while True:
                print("Waiting for TCP connection...")
                conn, addr = self.s.accept()
                print("Connected... Starting getting tweets.")
                sendData(conn, tags)
                conn.close()
        except KeyboardInterrupt:
            exit


if __name__ == '__main__':
    client = twitter_client("localhost", 9001)
    client.run_client(tags)

Waiting for TCP connection...
Connected... Starting getting tweets.
TEXT:RT @dpshaju: @PritishNandy Names of the people arrested in Indore for pelting stones at health officials who had gone for #coronavirus test…

TEXT:RT @kylegriffin1: Asked about McConnell blaming impeachment on being unprepared for coronavirus, Pelosi tells @MSNBC, "either you can't han…

TEXT:RT @nypost: Hydroxychloroquine rated 'most effective' coronavirus treatment, poll of doctors finds https://t.co/qjS8f0EXiv https://t.co/02r…

TEXT:RT @javimorgado: New: America’s top infectious disease expert, Dr. Anthony Fauci, said moments ago on CNN that the ENTIRE country should be…

TEXT:RT @h_tejas: Narendra Modi did not utter a single word about coronavirus cases, deaths, testing, health infrastructure, migrant workers, fo…

TEXT:RT @CBSLA: INSULT TO INJURY: 3 men were caught on video ransacking a Koreatown restaurant that was already understaffed and struggling beca…

TEXT:RT @sentiyapaa: Masterstroke by PM Modi. One 

TEXT:Those who attacked our doctors, health workers, nurses are enemies of the nation. Just think for a moment, if these… https://t.co/0aZcgnqOCs

TEXT:RT @bruce_arthur: They have done almost literally everything wrong

TEXT:RT @OH_mes2: Yuehua Entertainment confirms one of their staff members was diagnosed with Coronavirus

Everglow were tested and all came out…

TEXT:RT @OH_mes2: Yuehua Entertainment confirms one of their staff members was diagnosed with Coronavirus

Everglow were tested and all came out…

TEXT:nvm the app isn’t working i hate it here

TEXT:RT @Lrihendry: Just for the record, President Trump is sending almost all the stockpiled supplies to Democrat Governor hot spot cities. Doe…

TEXT:RT @NDIS: Want some tips on how to clean assistive equipment during this time? 

This advice might be helpful 👉 https://t.co/DxDHbX9aVv
#Co…

TEXT:RT @JanaSenaParty: We request everyone to donate a minimum amount of ₹100 and maximum upto your capacity to 'PM-Cares' inorder to support P…

TEXT:RT @spectatorindex: JUST IN: US reports a thousand coronavirus deaths in a 24-hour period for the first time

TEXT:RT @KamalaHarris: Fifty days ago I attended a Homeland Security committee meeting about the coronavirus—and no current Trump administration…

TEXT:‘We don’t have a lot of time’: Health expert gives stark warning to metro cities over #coronavirus #shtf #preppers… https://t.co/R3mOyIVwpS

TEXT:RT @SaraCarterDC: U.S. Coast Guard Seizes 19,000 Pounds Of Cocaine, Apprehend Smugglers 

“We may be getting more than narcotics because th…

TEXT:RT @ryanstruyk: Reported US coronavirus deaths via @CNN:

4 weeks ago: 12 deaths
3 weeks ago: 41 deaths
2 weeks ago: 195 deaths
1 week ago:…

TEXT:RT @briebriejoy: Yesterday, a Warren surrogate shared an alt-right meme denigrating Black female Bernie staffers, &amp; today, a Biden surrogat…

TEXT:RT @M2Madness: Every time someone dies from Coronavirus, a Democrat gets its wings... https://t.co/Cy3m0FGPxX

TEXT:RT @CaponesLady: 3Μ Michae

TEXT:RT @CNN: First responders in New York who cannot find or restart a pulse in adult cardiac arrest patients have been told not to bring those…

TEXT:RT @politicalmiller: SCHUMER wanted #coronavirus designated a public health emergency on Jan 26

BIDEN wrote an op-ed warning we’re not pre…

TEXT:RT @warmfourloko: well damn jackie i can’t control the coronavirus

TEXT:Idk why I barely just watched The Foreigner.. but it was a good ass movie

TEXT:I hate to say this, since i'm immunofucked, but if i was 99, i would go party too. How much life am i gonna lose at… https://t.co/mBFwoCDL4V

TEXT:RT @zacharybasu: Fauci calls for federally mandated stay-at-home order: "I don't understand why that's not happening.” https://t.co/kulK1X5…

TEXT:‘Rona on my mind

TEXT:RT @oye_its_simm_u: #SelflessServiceByDSSVolunteers

Coronavirus
@derasachasauda  volunteers of Nabha and Rajpura district Patiala Punjab t…

TEXT:RT @soledadobrien: The CDC, Centers for Disease Control, is located in his state.

T

TEXT:RT @WashTimes: Hydroxychloroquine rated 'most effective therapy' by doctors for coronavirus: Global survey https://t.co/rnshJqzP4O

TEXT:RT @WashTimes: Hydroxychloroquine rated 'most effective therapy' by doctors for coronavirus: Global survey https://t.co/rnshJqzP4O

TEXT:RT @KamalaHarris: Fifty days ago I attended a Homeland Security committee meeting about the coronavirus—and no current Trump administration…

TEXT:@RexChapman Day 41: in ER with broken neck and a fresh case of #coronavirus

TEXT:RT @OH_mes2: Yuehua Entertainment confirms one of their staff members was diagnosed with Coronavirus

Everglow were tested and all came out…

TEXT:RT @iconicakki: #coronavirus on 5th April at 9 P.M. https://t.co/ddfkmMlkR6

TEXT:RT @RBReich: Jeff Bezos' LA mansion has:

—3 saunas
—2 guesthouses
—A golf course
—A tennis court
—A "motor court" with gas pumps

His DC m…

TEXT:RT @seanhannity: ‘COMFORT’ ARRIVES: US Navy Hospital Ship Arrives in New York Harbor to Help Deal with Coronavirus h

...

TEXT:I think the horse has bolted. American leadership has been in decline since 2003. Trump is the death knell of US mo… https://t.co/wGbbQwHhE4

TEXT:RT @Brightstarll: What a movie ! 🥺💔

 — #miracleincellno7 https://t.co/8J00bWQSZa

TEXT:RT @JonKasdan: I wish I could go to a movie theater. Big auditorium, late show, mostly empty. Haven’t been much this year, busy with work,…

TEXT:Cruel irony: coronavirus has led the company that makes Corona beer to "suspend operations".

TEXT:RT @sardesairajdeep: Fine piece by @arvindsubraman and Devesh Kapur: why in every crisis there is an opportunity too. https://t.co/sk4bHf7o…

TEXT:RT @WHNSC: We thank the people of Taiwan for their generous support and collaboration as we continue our fight against the coronavirus pand…

TEXT:RT @doamuslims: Two Muslim doctors perform Salatul Janazah (funeral prayer) of someone who passed away from #coronavirus. 

#COVID19 https:…

TEXT:Top story: Coronavirus: Indonesia forgoes a ban on annual exodus to preven

TEXT:RT @cobbo3: In Africa, Fierce Enforcement of Coronavirus Lockdowns Is Stirring Resentment. Fears Are Growing in South Africa, Kenya and Uga…

TEXT:RT @jamieclimate: Fossil fuel companies are continuing pipeline construction even though it could spread the coronavirus, threatening the h…

TEXT:Coronavirus NSW: Hundreds hit Manly Beach amid social distancing restrictions https://t.co/vVrdHjhRkq #Sydney

TEXT:RT @SleepDeez: Can’t believe some of you haven’t seen Train To Busan. I’m two seconds from organizing a quarantined virtual movie night.

TEXT:What the hell you think was going to happen when you let him out? #coronavirus

TEXT:RT @astroehlein: The statement by EU member state leaders about Hungary’s coronavirus coup - a statement that didn’t even mention Hungary!…

TEXT:RT @Anaya56170931: #SelflessServiceByDSSVolunteers
Today, when the entire country is lockdown due to the coronavirus epidemic, poor familie…

TEXT:RT @anadoluagency: #BREAKING US coronavirus deaths surpass 6,000

TEXT:RT @Classic105Kenya: Maina kageni: Everyone had a life before Coronavirus happened but now things have changed. 

What do you miss most abo…

TEXT:Commander of confusion: Trump sows uncertainty and seeks to cast blame in coronavirus crisis https://t.co/XDT2kwxUpq

TEXT:Guess India is two weeks behind Italy in the Corona Outbreak curve...

TEXT:RT @OH_mes2: Yuehua Entertainment confirms one of their staff members was diagnosed with Coronavirus

Everglow were tested and all came out…

TEXT:Coronavirus Pandemic: Shana Casale, Emergency room nurse - https://t.co/oSVfS18Rfx - - #EnvironmentNews https://t.co/KQEBKAwfRA

TEXT:RT @ogundamisi: BREAKING: Osun records six new coronavirus cases
Total number of cases in Osun State is now 20. https://t.co/I1Q9Vaba04 via…

TEXT:RT @zachdcarter: Our leaders are failing us. https://t.co/6BGK6mIuJ2

TEXT:Top story: @CarlinhosVP Trump’s Company Seeks to Ease Financial Crunch as Coronavirus Takes Toll… https://t.co/3L4y8lvKYq

Error on_data: 'text'
W

TEXT:RT @Breaking911: “SHOOT THEM DEAD”: Philippine President Duterte threatens coronavirus lockdown violators. "If you are detained, I will lea…

TEXT:RT @stephlynn1982: Billionaires are fucking morons. They can’t be trusted with such wealth and power, time for forcibly remove it.🤣🤣

TEXT:RT @CNN: Former Labor Secretary Robert Reich says the coronavirus relief stimulus package, which includes sending checks to people, is "bar…

TEXT:RT @Scavino45: Coronavirus (COVID-19): Small Business Guidance &amp; Loan Resources⬇️⬇️ 
https://t.co/ucjfHleFDz

TEXT:RT @RealEricCarmen: Please retweet if you can. @realDonaldTrump @DonaldJTrumpJr @RealJamesWoods @TuckerCarlson 
@IngrahamAngle @greggutfeld…

TEXT:RT @johnlegend: "Kushner has succeeded at exactly three things in his life. He was born to the right parents, married well and learned how…

TEXT:RT @lc1_summit: I have finally recovered from the #Covid19 virus. 

I was so terribly ill.... 16 days of fever, 1 trip to ER. 

Stay safe.…

TEXT:RT @

Error on_data: 'text'
Waiting for TCP connection...
Connected... Starting getting tweets.
420
Waiting for TCP connection...
